## Beer Scrape with Pandas

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
from selenium import webdriver
import time
from sqlalchemy import create_engine




### Read in csv to pandas df

In [2]:
csv_data = pd.read_csv('CSV/recipeData.csv', encoding='latin1')
csv_data.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,1.038,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz,116.0
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,1.070,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,955.0
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,NaN,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN,NaN
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,NaN,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,NaN
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,1.050,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0


In [4]:
# Drop last five columns 

data = csv_data.drop(['PitchRate', 'PrimaryTemp', 'PrimingMethod', 'PrimingAmount', 'UserId'], axis=1)

pd.to_numeric(data['StyleID'])

# Create empty columns for values to add

data['ViewCount'] = ''
data['BrewCount'] = ''
data['LastUpdated'] = ''

list(data)

['BeerID',
 'Name',
 'URL',
 'Style',
 'StyleID',
 'Size(L)',
 'OG',
 'FG',
 'ABV',
 'IBU',
 'Color',
 'BoilSize',
 'BoilTime',
 'BoilGravity',
 'Efficiency',
 'MashThickness',
 'SugarScale',
 'BrewMethod',
 'ViewCount',
 'BrewCount',
 'LastUpdated']

In [5]:
# Color Range

max_color = np.nanmax(data.iloc[:, 10].values)
min_color = np.nanmin(data.iloc[:, 10].values)

f'Max Color: {max_color}, Min Color: {min_color}'

'Max Color: 186.0, Min Color: 0.0'

In [6]:
# View Count Range


In [ ]:
# Brew Count Range

In [7]:

data.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,ViewCount,BrewCount,LastUpdated
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,28.39,75,1.038,70.0,NaN,Specific Gravity,All Grain,,,
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,24.61,60,1.070,70.0,NaN,Specific Gravity,All Grain,,,
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,22.71,60,NaN,70.0,NaN,Specific Gravity,extract,,,
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,26.50,60,NaN,70.0,NaN,Specific Gravity,All Grain,,,
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,60.00,90,1.050,72.0,NaN,Specific Gravity,All Grain,,,


### Scrape individual beer page
### Grab view count, brew count, and Date
### Add to dataframe


In [8]:
# Scrape Setup

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

view_count = {}
brew_count = {}
updated_date = {}

driver = webdriver.Chrome() 

#executable_path = {'executable_path': driver}
browser = Browser('chrome')


for index, row in data.iterrows():
    
    try: 

        url = 'https://www.brewersfriend.com' + row['URL']    

        browser.visit(url)

        # Beautiful Soup Parsing
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        dates_scrape = soup.find('div', class_='lastupdated').text

        # View Count
        counts = soup.find('div', class_='lastupdated').find_all('b')

        view_count_str = counts[0].text
        brew_count_str = counts[1].text


        # Split view count to string by space, only keep second list item
        split_view_count = view_count_str.split(' ')
        view_count = int(split_view_count[2])

        # Split brew count to string by space, only keep second list item
        brew_view_count = brew_count_str.split(' ')
        brew_count = int(brew_view_count[2])



        # Add counts to data frame
        data.loc[index, 'ViewCount'] = view_count
        data.loc[index, 'BrewCount'] = brew_count

        # Grab date
        dates_scrape = soup.find('div', class_='lastupdated').text
        split_dates = dates_scrape.split(' ')
        date = split_dates[9]
        data.loc[index, 'LastUpdated'] = date
        
    except:
        pass

    


In [9]:
# Convert to datetime

data['LastUpdated'] = pd.to_datetime(data['LastUpdated'], format='%Y-%m-%d')
data


,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,ViewCount,BrewCount,LastUpdated
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,28.39,75,1.038,70.0,NaN,Specific Gravity,All Grain,150484,42,2016-06-14
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,24.61,60,1.070,70.0,NaN,Specific Gravity,All Grain,134371,54,2018-01-04
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,22.71,60,NaN,70.0,NaN,Specific Gravity,extract,133892,119,2018-03-02
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,26.50,60,NaN,70.0,NaN,Specific Gravity,All Grain,134161,91,2018-01-04
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,60.00,90,1.050,72.0,NaN,Specific Gravity,All Grain,131887,25,2017-01-13
5,6,Sierra Nevada Pale Ale Clone,/homebrew/recipe/view/28546/sierra-nevada-pale...,American Pale Ale,10,24.61,1.055,1.013,5.58,40.12,...,29.34,70,1.047,79.0,NaN,Specific Gravity,All Grain,121823,302,2018-03-04
6,7,Russian River Pliny the Elder (original),/homebrew/recipe/view/37534/russian-river-plin...,Imperial IPA,86,22.71,1.072,1.018,7.09,268.71,...,30.28,90,NaN,75.0,NaN,Specific Gravity,All Grain,121959,28,2016-11-29
7,8,Spotted Clown (New Glarus Spotted Cow clone),/homebrew/recipe/view/672/spotted-clown-new-gl...,Cream Ale,45,20.82,1.054,1.014,5.36,19.97,...,28.39,75,1.040,70.0,1.40,Specific Gravity,All Grain,95189,18,2017-07-30
8,9,Chocolate Vanilla Porter,/homebrew/recipe/view/29265/chocolate-vanilla-...,Robust Porter,129,22.71,1.060,1.016,5.77,31.63,...,30.28,75,1.042,73.0,NaN,Specific Gravity,All Grain,89406,22,2018-03-09
9,10,Mango Habanero IPA,/homebrew/recipe/view/61082/mango-habanero-ipa,Imperial IPA,86,20.82,1.080,1.017,8.22,93.02,...,28.39,60,1.058,70.0,NaN,Specific Gravity,All Grain,81189,24,2017-12-08


In [10]:
engine = create_engine('sqlite:///data.db')

data.to_sql("data", con=engine, flavor="sqlite", if_exists="replace", index=False)


/Users/EthanCraddock/anaconda/envs/PythonData/lib/python3.6/site-packages/pandas/io/sql.py:527: FutureWarning: the 'flavor' parameter is deprecated and will be removed in a future version, as 'sqlite' is the only supported option when SQLAlchemy is not installed.
  _validate_flavor_parameter(flavor)


In [17]:
data.to_csv('100_test_data.csv', index=False)